In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import asyncio
import json
import os
import warnings
from typing import List
from pydantic import BaseModel, Field
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import Session
from google.genai import types
from google.genai.client import Client

print("Imported Successfully")

In [ ]:
# Suppress Python warnings
warnings.filterwarnings('ignore')

# Suppress logging warnings for google_genai
import logging
logging.getLogger('google_genai.types').setLevel(logging.ERROR)
logging.getLogger('google_genai').setLevel(logging.ERROR)
logging.getLogger('google.genai').setLevel(logging.ERROR)


In [ ]:
# ============================================================================
# AUTHENTICATION SETUP FOR KAGGLE
# ============================================================================
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

try:
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ API Key loaded successfully from Kaggle Secrets")
except Exception as e:
    print(f"❌ Error loading API key: {e}")
    print("⚠️  Please add GOOGLE_API_KEY to Kaggle Secrets")
    raise

client = Client(api_key=GOOGLE_API_KEY)
MODEL_NAME = "gemini-2.5-flash-lite"


In [ ]:
# ============================================================================
# PRODUCT DATABASE
# ============================================================================

ACME_PRODUCTS = [
    {
        "product_id": "ACME-FRG-001",
        "product_name": "ACME Smart Refrigerator",
        "description": "Premium 4-door French door refrigerator with smart features, water dispenser, and LED display. Stainless steel finish with 25 cu.ft. capacity.",
        "brand": "ACME",
        "search_tags": ["refrigerator", "fridge", "smart fridge", "french door", "kitchen appliance"],
        "price": "$2,499",
        "features": ["Smart features", "Water dispenser", "LED display", "25 cu.ft. capacity"]
    },
    {
        "product_id": "ACME-LED-001",
        "product_name": "ACME Color Changing Smart LED Bulb",
        "description": "16 million color smart LED bulb, app-controlled, dimmable, works with voice assistants. Energy efficient with WiFi connectivity.",
        "brand": "ACME",
        "search_tags": ["LED", "light bulb", "smart bulb", "color changing", "WiFi bulb", "smart home"],
        "price": "$29.99",
        "features": ["16 million colors", "App-controlled", "Voice assistant compatible", "Dimmable"]
    },
    {
        "product_id": "ACME-UMB-001",
        "product_name": "ACME Professional Umbrella",
        "description": "Large black umbrella with ACME branding, windproof design, automatic open, and durable construction with carrying case.",
        "brand": "ACME",
        "search_tags": ["umbrella", "rain protection", "windproof", "outdoor", "branded"],
        "price": "$49.99",
        "features": ["Windproof design", "Automatic open", "Includes carrying case", "ACME branded"]
    },
    {
        "product_id": "ACME-LAD-001",
        "product_name": "ACME Extension Ladder",
        "description": "Professional-grade aluminum extension ladder, 8-foot capacity, lightweight yet sturdy with slip-resistant feet and safety lock.",
        "brand": "ACME",
        "search_tags": ["ladder", "extension ladder", "aluminum ladder", "tool", "construction", "home improvement"],
        "price": "$159.99",
        "features": ["8-foot capacity", "Aluminum construction", "Slip-resistant feet", "Safety lock"]
    },
    {
        "product_id": "ACME-BLW-001",
        "product_name": "ACME Electric Leaf Blower",
        "description": "Powerful electric leaf blower in purple finish, corded design with variable speed control and ergonomic handle for yard maintenance.",
        "brand": "ACME",
        "search_tags": ["leaf blower", "leafblower", "blower", "yard tool", "garden tool", "electric blower"],
        "price": "$89.99",
        "features": ["Electric powered", "Variable speed control", "Ergonomic handle", "Purple finish"]
    }
]

print("Product Database Created")

In [ ]:
# ============================================================================
# AGENT TOOLS
# ============================================================================

def search_products(query: str) -> str:
    """
    Search ACME product database for relevant products.
    
    Args:
        query: Customer search query (e.g., 'smart home', 'kitchen', 'outdoor')
        
    Returns:
        JSON string with matching products
    """
    query_lower = query.lower()
    matching_products = []
        
    for product in ACME_PRODUCTS:
        search_text = (
            product["product_name"].lower() + " " +
            product["product_id"].lower() + " " +
            product["description"].lower() + " " +
            " ".join(product["search_tags"])
        )
        
        query_words = [w.strip() for w in query_lower.split() if len(w.strip()) > 2]
        if any(word in search_text for word in query_words):
            matching_products.append(product)
    
    if not matching_products:
        return json.dumps({
            "message": "No products found matching your query.", 
            "products": [],
            "suggestion": "Try: refrigerator, LED bulb, umbrella, ladder, or leaf blower"
        })
         
    return json.dumps({
        "found": len(matching_products),
        "products": matching_products
    }, indent=2)

def get_product_details(product_identifier: str) -> str:
    """
    Get detailed information about a specific ACME product.
    
    Args:
        product_identifier: Product name, ID, or keyword
        
    Returns:
        JSON string with complete product details
    """
    identifier_lower = product_identifier.lower()
    
    for product in ACME_PRODUCTS:
        # Check product ID
        if identifier_lower in product["product_id"].lower():
            return json.dumps({"success": True, "product": product}, indent=2)
        
        # Check product name (flexible word matching)
        name_words = product["product_name"].lower().split()
        query_words = identifier_lower.split()
        if any(qw in name_words for qw in query_words if len(qw) > 2):
            return json.dumps({"success": True, "product": product}, indent=2)
        
        # Check search tags
        if any(identifier_lower in tag.lower() for tag in product["search_tags"]):
            return json.dumps({"success": True, "product": product}, indent=2)
    
    return json.dumps({
        "success": False,
        "error": f"No product found matching '{product_identifier}'.",
        "available_products": [p["product_name"] for p in ACME_PRODUCTS]
    })
    
def list_all_products() -> str:
    """List all available ACME products with prices."""
    product_list = [
        {
            "name": p["product_name"],
            "id": p["product_id"],
            "price": p["price"],
            "category": p["search_tags"][0],
            "description": p["description"]
        }
        for p in ACME_PRODUCTS
    ]
    
    return json.dumps({
        "total_products": len(ACME_PRODUCTS),
        "products": product_list
    }, indent=2)


In [ ]:
# ============================================================================
# MAIN ASYNC FUNCTION
# ============================================================================
async def main():
    """Run the ACME customer support agent."""
    
    print("\n" + "="*80)
    print("🏢 ACME CUSTOMER SUPPORT AGENT")
    print("="*80)
    print("🤖 Powered by Google ADK and Gemini AI")
    print("📦 5 Products Available | 💬 Text-Only Responses")
    print(f"🔑 Model: {MODEL_NAME}")
    print("="*80 + "\n")
    
    app_name = 'acme_support_agent'
    user_id = 'customer_001'
    
    support_agent = Agent(
        model=MODEL_NAME,
        name="acme_support_specialist",
        instruction="""You are a friendly ACME customer support specialist.

IMPORTANT RULES:
1. When a customer mentions a product name, use get_product_details to fetch information
2. If the tool returns success, present the product details naturally
3. Always be helpful, clear, and avoid contradicting yourself
4. Include prices and key features when describing products

YOUR TOOLS:
- search_products: Find products by keywords (kitchen, smart home, outdoor, tools)
- get_product_details: Get complete details about a specific product
- list_all_products: Show all available ACME products

ACME PRODUCT CATALOG:
- ACME Smart Refrigerator ($2,499) - Premium kitchen appliance
- ACME Color Changing Smart LED Bulb ($29.99) - Smart home lighting
- ACME Professional Umbrella ($49.99) - Weather protection
- ACME Extension Ladder ($159.99) - Professional tool
- ACME Electric Leaf Blower ($89.99) - Yard maintenance

Be conversational and use the tools to provide accurate product information!""",
        tools=[search_products, get_product_details, list_all_products],
        disallow_transfer_to_parent=True,
        disallow_transfer_to_peers=True,
    )
    
    runner = InMemoryRunner(agent=support_agent, app_name=app_name)
    session = await runner.session_service.create_session(app_name=app_name, user_id=user_id)
    
    async def run_query(session: Session, user_message: str):
        """Process customer query."""
        content = types.Content(
            role='user', 
            parts=[types.Part.from_text(text=user_message)]
        )
        
        print(f"\n{'─'*80}")
        print(f"💬 YOU: {user_message}")
        print(f"{'─'*80}")
        print("🤖 ACME SUPPORT: ", end="", flush=True)
        
        async for event in runner.run_async(user_id=user_id, session_id=session.id, new_message=content):
            if event.content.parts and event.content.parts[0].text:
                chunk = event.content.parts[0].text
                print(chunk, end="", flush=True)
        print()
        
    # INTERACTIVE LOOP
    print("\n" + "="*80)
    print("🎯 INTERACTIVE MODE ACTIVATED")
    print("="*80)
    print("💡 Example queries:")
    print("   - 'Show me all products'")
    print("   - 'ACME Smart Refrigerator'")
    print("   - 'I need something for my kitchen'")
    print("   - 'Tell me about the LED bulb'")
    print("   - 'Do you have outdoor products?'")
    print("\n🚪 Type 'exit', 'quit', or 'bye' to end conversation")
    print("="*80)
    
    await run_query(session, "Hello! What products does ACME offer?")
    
    while True:
        print("\n" + "─"*80)
        try:
            user_input = await asyncio.get_event_loop().run_in_executor(
                None, lambda: input("💬 YOU: ")
            )
            
            if user_input.lower().strip() in ['exit', 'quit', 'q', 'bye', 'goodbye']:
                print("\n" + "="*80)
                print("👋 Thank you for contacting ACME Customer Support!")
                print("🌟 Have a great day!")
                print("="*80 + "\n")
                break
            
            if not user_input.strip():
                print("⚠️  Please enter a question or type 'exit' to quit.")
                continue
            
            await run_query(session, user_input.strip())
            
        except KeyboardInterrupt:
            print("\n\n⚠️  Conversation interrupted by user.")
            print("👋 Goodbye!\n")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("🔄 Please try again or type 'exit' to quit.\n")


In [ ]:
# ============================================================================
# RUN THE AGENT
# ============================================================================

await main()